# import modules

In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

# Data set 불러오기

In [6]:
X_df=pd.read_excel('C:/Statistics/2020bigcon_train.xlsx',header = 1)

In [7]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 하세요! 그게 제일 빨라요
X = X_df.copy()

In [8]:
X.head() 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


In [9]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


# 20200805 결측치 처리 과정

In [10]:
X.isnull().sum()

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액        937
dtype: int64

## 상품군 곁측치 처리

 판매단가가 0 인 상품은 예측에서 제외

In [11]:
def NA_goods(df=X):
    df=df.loc[df['판매단가'] != 0,:]
    df = df.reset_index(drop=True)
    return df

In [12]:
NA_goods(X)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0
37369,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0
37370,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0


## 노출(분) 곁측치 처리

노출분은 같은 시간에 같이 방송되어서 NaN 으로 뜨는것이다. <br>
아마 같은 시간에 방송되어진 값을 평균해서 채우면 될듯?

In [13]:
# 0 노출시간이 0 인 데이터가 없다.
# 즉 Na 처리할떄 0으로 대채한 후에 평균떄려도 아무 상관 없다는 이야기!
(X.loc[:,'노출(분)']==0).sum()

0

In [14]:
# 노출(분) 을 제일 먼저 시작한 시간의 평균으로 만드는 FUNCTION 정의
# 아직 맞는지는 잘 몰라서 안맞는 경우 수정해주세요.
def NA_minute(df=X):
    # 곁측치를 0으로 대체
    X['노출(분)'] = X['노출(분)'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'노출(분)'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'노출(분)'] = df.loc[start,'노출(분)'] / num 
    return df

In [15]:
NA_minute(X)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,10.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,10.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,10.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,10.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,10.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


## 곁측치 처리 pipeline

In [16]:
def Na_pipeline(df = X):
    df = NA_goods(df)
    df = NA_minute(df)
    return(df)

In [17]:
X = Na_pipeline(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,5.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
37368,2020-01-01 00:00:00,5.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0
37369,2020-01-01 00:00:00,5.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0
37370,2020-01-01 00:00:00,5.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0


In [18]:
# na 없다
X.isnull().sum().sum()

0

# 데이터 살펴보기

노출 분은 대부분 10 분 내외인듯.

In [19]:
pd.DataFrame(pd.Series(X['노출(분)']).value_counts(normalize=True))

,노출(분)
20.000000,0.299502
10.000000,0.225222
5.000000,0.157284
3.333333,0.096329
6.666667,0.074039
...,...
11.033333,0.000027
13.500000,0.000027
17.083333,0.000027
19.150000,0.000027


# 20200807 UPDATE 결측치 처리 과정

매주 토요일 18:00~18:20은 정보방송시간으로 추정 제외(당사사정에 따라 ±20 편성 변경 있음)

In [20]:
t = ['월','화','수','목','금','토','일']
import datetime
r = datetime.datetime.today()
r

datetime.datetime(2020, 8, 8, 2, 35, 35, 599717)

In [21]:
def printDayOfTheWeek(year, month, day):
    dayOfTheWeek = ['월','화','수','목','금','토','일']
    return dayOfTheWeek[datetime.datetime(year,month,day).weekday()]

In [22]:
import time
day_list = []
for date in X.iloc[:,0]:
    date = str(date)
    year = int(date[:4])
    month = int(date[5:7])
    day = int(date[8:10])
    first = printDayOfTheWeek(year,month,day)
    second = date[11:16]
    day_list.append(first+second)

In [23]:
X["요일/시간"] = day_list

In [24]:
X.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,요일/시간
0,2019-01-01 06:00:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,화06:00
1,2019-01-01 06:00:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,화06:00
2,2019-01-01 06:20:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,화06:20
3,2019-01-01 06:20:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,화06:20
4,2019-01-01 06:40:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,화06:40


In [25]:
for i in range(len(X)):
    time = X.iloc[i,8]
    if (time[0:3]=="토18" and int(time[4:])<20):
        X = X.drop(i)

In [26]:
len(X)

37372

In [27]:
for i in range(len(X)):
    time = X.iloc[i,8]
    if time[0:3]=="토18":
        print(time)

토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:30
토18:40
토18:50
토18:20
토18:50
토18:20
토18:30
토18:40
토18:50
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:20
토18:20
토18:20
토18:20
토18:20
토18:40
토18:40
토18:40
토18:40
토18:40
토18:40


힘들게 코드를 짰으나 토요일 18:00~18:20에 해당하는 방송은 없었다.............................

# 변수 늘리기

## 시간 데이터 늘리기

In [37]:
# 시각은 우선 시,분을 합쳐서 하나의 값으로 만들었습니다.
def time_feature(df = X):
    df['방송일시']=df['방송일시'].astype('str')
    df['날짜']=df['방송일시'].str.split(' ').str[0]
    df['시간']=df['방송일시'].str.split(' ').str[1]
    df['년']=df['날짜'].str.split('-').str[0]
    df['월']=df['날짜'].str.split('-').str[1]
    df['일']=df['날짜'].str.split('-').str[2]
    df['시']=df['시간'].str.split(':').str[0]
    df['분']=df['시간'].str.split(':').str[1]
    df['초']=df['시간'].str.split(':').str[2]
    df['시각'] = df['시'].astype('float32') + df['분'].astype('float32')*(1/60)
    df.drop(['시','분','초','시간'],axis=1,inplace = True)
    return(df)

In [38]:
X = time_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,요일/시간,년,월,일,시각,점심,저녁,날짜
0,2019-01-01 06:00:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,화06:00,2019,01,01,6.000000,0,0,2019-01-01
1,2019-01-01 06:00:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,화06:00,2019,01,01,6.000000,0,0,2019-01-01
2,2019-01-01 06:20:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,화06:20,2019,01,01,6.333333,0,0,2019-01-01
3,2019-01-01 06:20:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,화06:20,2019,01,01,6.333333,0,0,2019-01-01
4,2019-01-01 06:40:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,화06:40,2019,01,01,6.666667,0,0,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,5.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,화23:40,2019,12,31,23.666667,0,0,2019-12-31
37368,2020-01-01 00:00:00,5.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01
37369,2020-01-01 00:00:00,5.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01
37370,2020-01-01 00:00:00,5.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01


## 식사 시간과 관련된 변수

식사시간의 경우 유의미한 데이터가 있을 수 있다. 그 시간대를 잡아서 변수를 늘리면 효과가 있을거같다. <br>
하지만 아침 식사시간의 경우 거르는 인구가 33%(통계청) 이나 되는 이유로, 오히려 혼란을 가중시킬 수 있기떄문에 그냥 점심과 저녁만 고려하자

In [39]:
def eat_feature(df = X):
    df['점심'] = 0
    df.loc[ (11.5<df['시각']) & (df['시각'] < 13),'점심']  = 1
    df['저녁'] = 0
    df.loc[ (17.5<df['시각'])&(df['시각']<19.5) , '저녁'] = 1
    return(df)

In [40]:
X = eat_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,요일/시간,년,월,일,시각,점심,저녁,날짜
0,2019-01-01 06:00:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,화06:00,2019,01,01,6.000000,0,0,2019-01-01
1,2019-01-01 06:00:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,화06:00,2019,01,01,6.000000,0,0,2019-01-01
2,2019-01-01 06:20:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,화06:20,2019,01,01,6.333333,0,0,2019-01-01
3,2019-01-01 06:20:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,화06:20,2019,01,01,6.333333,0,0,2019-01-01
4,2019-01-01 06:40:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,화06:40,2019,01,01,6.666667,0,0,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,5.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,화23:40,2019,12,31,23.666667,0,0,2019-12-31
37368,2020-01-01 00:00:00,5.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01
37369,2020-01-01 00:00:00,5.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01
37370,2020-01-01 00:00:00,5.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01


In [41]:
pd.Series(X['점심']).value_counts() # 좀 sparse 하긴 한데 의미는 있을거라고 믿는다...

0    35025
1     2347
Name: 점심, dtype: int64

# 날씨와 관련된 변수

날씨 데이터의 경우 기상청의 공공데이터를 사용했다.
일별 기온(평균기온, 최고기온, 최저기온), 일별 강수량, 월별 기상정보(폭염일수, 열대야일수)를 고려했다.

In [42]:
rain_df=pd.read_csv('C:/Statistics/19rain.csv',header = 7, encoding='CP949')
rain = rain_df.copy()

In [43]:
rain.head()

,날짜,지점,강수량(mm)
0,2019-01-01,전국,0.1
1,2019-01-02,전국,0.0
2,2019-01-03,전국,0.0
3,2019-01-04,전국,0.0
4,2019-01-05,전국,0.0


In [44]:
def rain_feature(df = X):
       
    df = pd.merge(df, rain, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)

X = rain_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,요일/시간,년,월,일,시각,점심,저녁,날짜,강수량(mm)
0,2019-01-01 06:00:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,화06:00,2019,01,01,6.000000,0,0,2019-01-01,0.1
1,2019-01-01 06:00:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,화06:00,2019,01,01,6.000000,0,0,2019-01-01,0.1
2,2019-01-01 06:20:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,화06:20,2019,01,01,6.333333,0,0,2019-01-01,0.1
3,2019-01-01 06:20:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,화06:20,2019,01,01,6.333333,0,0,2019-01-01,0.1
4,2019-01-01 06:40:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,화06:40,2019,01,01,6.666667,0,0,2019-01-01,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,5.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,화23:40,2019,12,31,23.666667,0,0,2019-12-31,0.1
37368,2020-01-01 00:00:00,5.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01,0.0
37369,2020-01-01 00:00:00,5.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01,0.0
37370,2020-01-01 00:00:00,5.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01,0.0


In [45]:
temp_df=pd.read_csv('C:/Statistics/19temp.csv',header = 7, encoding='CP949')
temp = temp_df.copy()

In [46]:
temp.head()

,날짜,지점,평균기온(℃),최저기온(℃),최고기온(℃)
0,2019-01-01,전국,-2.1,-5.8,2.1
1,2019-01-02,전국,-2.5,-7.0,3.2
2,2019-01-03,전국,-2.1,-7.7,5.0
3,2019-01-04,전국,-0.7,-7.3,5.3
4,2019-01-05,전국,0.2,-4.6,5.3


In [47]:
def temp_feature(df = X):
       
    df = pd.merge(df, temp, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)

X = temp_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,요일/시간,년,월,일,시각,점심,저녁,날짜,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃)
0,2019-01-01 06:00:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,화06:00,2019,01,01,6.000000,0,0,2019-01-01,0.1,-2.1,-5.8,2.1
1,2019-01-01 06:00:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,화06:00,2019,01,01,6.000000,0,0,2019-01-01,0.1,-2.1,-5.8,2.1
2,2019-01-01 06:20:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,화06:20,2019,01,01,6.333333,0,0,2019-01-01,0.1,-2.1,-5.8,2.1
3,2019-01-01 06:20:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,화06:20,2019,01,01,6.333333,0,0,2019-01-01,0.1,-2.1,-5.8,2.1
4,2019-01-01 06:40:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,화06:40,2019,01,01,6.666667,0,0,2019-01-01,0.1,-2.1,-5.8,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,5.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,화23:40,2019,12,31,23.666667,0,0,2019-12-31,0.1,-3.8,-6.7,0.4
37368,2020-01-01 00:00:00,5.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01,0.0,4.1,-6.5,4.1
37369,2020-01-01 00:00:00,5.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01,0.0,4.1,-6.5,4.1
37370,2020-01-01 00:00:00,5.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,수00:00,2020,01,01,0.000000,0,0,2020-01-01,0.0,4.1,-6.5,4.1


In [48]:
hot_df=pd.read_csv('C:/Statistics/19hot.csv',header = 4, encoding='CP949')
hot = hot_df.copy()

In [49]:
hot.head()

,월,평균 열대야일수,평균 폭염일수
0,1,0.0,0.0
1,2,0.0,0.0
2,3,0.0,0.0
3,4,0.0,0.0
4,5,0.0,0.1


In [50]:
def hot_feature(df = X):
    hot['월']=hot['월'].astype('str')   
    df = pd.merge(df, hot, on="월", how= "inner")
    return(df)

X = hot_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,요일/시간,년,...,시각,점심,저녁,날짜,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),평균 열대야일수,평균 폭염일수
0,2019-10-01 01:00:00,20.000000,100175,200594,바두기 무선 에어 바리캉,생활용품,69800,7907000.0,화01:00,2019,...,1.000000,0,0,2019-10-01,9.4,21.5,18.4,24.9,0.0,0.0
1,2019-10-01 01:20:00,20.000000,100175,200594,바두기 무선 에어 바리캉,생활용품,69800,15819000.0,화01:20,2019,...,1.333333,0,0,2019-10-01,9.4,21.5,18.4,24.9,0.0,0.0
2,2019-10-01 01:40:00,16.683333,100175,200594,바두기 무선 에어 바리캉,생활용품,69800,14032000.0,화01:40,2019,...,1.666667,0,0,2019-10-01,9.4,21.5,18.4,24.9,0.0,0.0
3,2019-10-01 06:00:00,20.000000,100603,201845,카파 FW 언더셔츠 6종 패키지,속옷,69000,6798000.0,화06:00,2019,...,6.000000,0,0,2019-10-01,9.4,21.5,18.4,24.9,0.0,0.0
4,2019-10-01 06:20:00,20.000000,100603,201845,카파 FW 언더셔츠 6종 패키지,속옷,69000,15500000.0,화06:20,2019,...,6.333333,0,0,2019-10-01,9.4,21.5,18.4,24.9,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,2019-12-31 23:20:00,5.000000,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,화23:20,2019,...,23.333333,0,0,2019-12-31,0.1,-3.8,-6.7,0.4,0.0,0.0
8834,2019-12-31 23:40:00,5.000000,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,화23:40,2019,...,23.666667,0,0,2019-12-31,0.1,-3.8,-6.7,0.4,0.0,0.0
8835,2019-12-31 23:40:00,5.000000,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,15282000.0,화23:40,2019,...,23.666667,0,0,2019-12-31,0.1,-3.8,-6.7,0.4,0.0,0.0
8836,2019-12-31 23:40:00,5.000000,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,화23:40,2019,...,23.666667,0,0,2019-12-31,0.1,-3.8,-6.7,0.4,0.0,0.0


## Feature 늘리기 Pipeline

# 데이터 type 처리

상품군에 대해서 onehot encoding 적용하자

아 그런데 시각 데이터에 대해서, 0 과 24에 가까운 값들은 비슷한 속성을 가짐에도(늦은시간) 값이 차이가 많이난다.
그러므로 내 생각에는 그냥 categorical 로 싹다 해버리는게 좋을거같은데..?